In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import konlpy
import mecab

In [2]:
file_path = '감성대화말뭉치(원시데이터)_Training.xlsx'
df = pd.read_excel(file_path)
df.shape

(74856, 14)

In [3]:
df.head()

,번호,value,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3
0,24070,1,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,지금 난 기분이 너무 좋아.,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.,그거 정말 기분 좋을 만하네요. 무엇을 할 예정인가요?,그냥 집에서 뒹굴뒹굴하고 넷플릭스 보고 놀 거야.,집에서 노는 것이 최고죠. 생각만 해도 행복하겠어요.
1,27879,1,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,NaN,NaN
2,51471,1,중년,여성,"직장, 업무 스트레스",해당없음,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,NaN,NaN
3,7140,1,노년,남성,대인관계,만성질환 유,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.,지금의 기분에서 벗어나기 위해 무엇을 하면 좋을까요?,아내에게 졸혼은 어떤지 대화를 나누어 봐야겠어. 졸혼을 한다면 가끔씩이라도 도움을 ...,아내와의 대화로 기분이 나아졌으면 좋겠어요.
4,39663,1,노년,남성,재정,만성질환 유,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,NaN,NaN


In [4]:
df.isna().sum()

번호            0
value         0
연령            0
성별            0
상황키워드         0
신체질환          0
감정_대분류        0
감정_소분류        0
사람문장1         0
시스템응답1        0
사람문장2         0
시스템응답2        1
사람문장3     25196
시스템응답3    25202
dtype: int64

In [5]:
df.drop(['사람문장2', '사람문장3','시스템응답3'], axis=1, inplace=True)
df.shape

(74856, 11)

In [6]:
df.dropna(axis=0, inplace=True)
df.shape

(74855, 11)

In [7]:
df.duplicated().sum()

0

In [8]:
df.dtypes

번호         int64
value      int64
연령        object
성별        object
상황키워드     object
신체질환      object
감정_대분류    object
감정_소분류    object
사람문장1     object
시스템응답1    object
시스템응답2    object
dtype: object

In [9]:
df['value'].value_counts()

1    74855
Name: value, dtype: int64

In [10]:
df.drop(['번호', 'value'], axis=1, inplace=True)
df.shape

(74855, 9)

In [11]:
df['신체질환'].value_counts()

해당없음       56467
만성질환 유      9246
만성질환 무      9140
만성질환 무         2
Name: 신체질환, dtype: int64

In [12]:
df.columns = ['Age', 'Gender', 'Situation', 'Condition', 'Emotion1', 'Emotion2', 'Human1', 'Computer1', 'Computer2']

In [13]:
df.head()

,Age,Gender,Situation,Condition,Emotion1,Emotion2,Human1,Computer1,Computer2
0,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,지금 난 기분이 너무 좋아.,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?,그거 정말 기분 좋을 만하네요. 무엇을 할 예정인가요?
1,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,언니에게 꽃을 선물하려고 하시는군요.
2,중년,여성,"직장, 업무 스트레스",해당없음,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,계약을 성공적으로 마무리하실 수 있기를 기원해요.
3,노년,남성,대인관계,만성질환 유,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.,지금의 기분에서 벗어나기 위해 무엇을 하면 좋을까요?
4,노년,남성,재정,만성질환 유,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.


In [14]:
df.Emotion1.value_counts()

상처    13504
슬픔    12708
분노    12503
기쁨    12449
불안    12383
당황    11308
Name: Emotion1, dtype: int64

In [18]:
features = df[['Human1']]
targets = df[['Emotion1']] 

features.shape, targets.shape

((74855, 1), (74855, 1))

In [19]:
features.head()

,Human1
0,지금 난 기분이 너무 좋아.
1,나도 조카가 생겨! 너무 기뻐.
2,거래처와의 다음 계약이 무산될까봐 불안해.
3,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...
4,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.


In [20]:
targets.head()

,Emotion1
0,기쁨
1,기쁨
2,불안
3,슬픔
4,슬픔


In [22]:
with open('한국어불용어100.txt') as f:
    lines = f.readlines()
stopwords = []
for stopword in lines:
    stopwords.append(stopword.split('\t')[0])
print(stopwords)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']


In [26]:
features['Human1'] = features['Human1'].str.replace("[^ ㄱ-ㅣ가-힣a-zA-Z0-9+]", " ")
features.head()

/var/folders/9_/rnj5svk11p1157tkdv28yb6r0000gn/T/ipykernel_44099/528249083.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  features['Human1'] = features['Human1'].str.replace("[^ ㄱ-ㅣ가-힣a-zA-Z0-9+]", " ")
/var/folders/9_/rnj5svk11p1157tkdv28yb6r0000gn/T/ipykernel_44099/528249083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Human1'] = features['Human1'].str.replace("[^ ㄱ-ㅣ가-힣a-zA-Z0-9+]", " ")


,Human1
0,지금 난 기분이 너무 좋아
1,나도 조카가 생겨 너무 기뻐
2,거래처와의 다음 계약이 무산될까봐 불안해
3,당뇨로 고생 중인데 아내가 이혼하자더군 아내가 날 버리면 병은 더 악화될 텐데 세...
4,노년에 재정적으로 이루어 놓은게 없어서 걱정이야


In [51]:
features['Human1'] = features['Human1'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))

/var/folders/9_/rnj5svk11p1157tkdv28yb6r0000gn/T/ipykernel_44099/3590510365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Human1'] = features['Human1'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))


In [52]:
features.head()

,Human1
0,난 기분이 너무 좋아
1,나도 조카가 생겨 너무 기뻐
2,거래처와의 다음 계약이 무산될까봐 불안해
3,당뇨로 고생 중인데 아내가 이혼하자더군 아내가 날 버리면 병은 악화될 텐데 세상에 ...
4,노년에 재정적으로 이루어 놓은게 없어서 걱정이야


In [53]:
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder

enc = OrdinalEncoder()
target = enc.fit_transform(targets['Emotion1'])
X_train, X_test, y_train, y_test = train_test_split(features['Human1'], target, 
                                                    test_size=0.15, stratify=target,
                                                    random_state=33)

X_train.shape, X_test.shape

((63626,), (11229,))

In [54]:
X_train.head()

10349    오늘 길 가는데 모르는 사람이 인사를 하는 거야
44343                 요새 죄책감이 많이 들어
2590            사람만 생각하면 구역질이나고 화가나
54945             아들이랑 나는 사이가 정말 좋아
41867              미래에 뭘 해야 할지 모르겠어
Name: Human1, dtype: object

In [55]:
y_train['Emotion1'] = y_train['Emotion1'].apply(lambda x: x-1)
y_train.head()

,Emotion1
10349,3
44343,3
2590,5
54945,0
41867,4


In [56]:
y_test['Emotion1'] = y_test['Emotion1'].apply(lambda x: x-1)
y_test.head()

,Emotion1
27233,2
50477,2
25256,4
24432,0
1177,1


In [57]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, LSTM, Embedding, SimpleRNN, RNN, LSTMCell
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

y_train_oh = tf.one_hot(y_train['Emotion1'], 6)
y_test_oh = tf.one_hot(y_test['Emotion1'], 6)
y_train_oh

<tf.Tensor: shape=(63626, 6), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)>

In [58]:
token = Tokenizer(num_words=6000)

In [59]:
token.fit_on_texts(X_train)
len(token.word_index)

64303

In [60]:
X_train_enc = token.texts_to_sequences(X_train)

In [61]:
max(len(x) for x in X_train_enc)

26

In [62]:
np.mean([len(x) for x in X_train_enc])

6.109294942319178

In [63]:
X_train = sequence.pad_sequences(X_train_enc, maxlen=15, padding='post')
X_train

array([[   5, 3128,  814, ...,    0,    0,    0],
       [ 139,  358,   11, ...,    0,    0,    0],
       [5809,  326,  468, ...,    0,    0,    0],
       ...,
       [   6,   71, 2393, ...,    0,    0,    0],
       [ 219, 4212,  732, ...,    0,    0,    0],
       [2087,   29, 4152, ...,    0,    0,    0]], dtype=int32)

In [64]:
vocab_size = len(token.word_index) + 1
vocab_size

64304

In [99]:
import IPython
import keras_tuner as kt
from tensorflow import keras

tf.random.set_seed(33)

def model_builder(hp):
    
    output_size = 6
    hp_units = hp.Int('units', min_value = 40, max_value = 96, step = 8)
    hp_units1 = hp.Int('units1', min_value = 40, max_value = 96, step = 8)
    hp_units2 = hp.Int('units2', min_value = 40, max_value = 96, step = 8)
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2]) 
    
    lstm_layer = keras.layers.LSTM(hp_units, 
                                   activation='tanh', 
                                   recurrent_activation='sigmoid')
    
    model = keras.models.Sequential(
        [
            keras.layers.Input(shape=(15,)),
            keras.layers.Embedding(vocab_size, 64, input_length=15),
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(hp_units1, activation='relu'),
            keras.layers.Dense(hp_units2, activation='relu'),
            keras.layers.Dense(output_size, activation='softmax'),
        ]
    )
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  metrics=['accuracy'])
    return model

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 100,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'kt9_LSTM_result')

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                           min_delta=0, 
                                           patience=5, 
                                           verbose=1)

checkpoint_filepath = "best_a.hdf5"
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, 
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True,
    save_weights_only=True, 
    mode='auto', 
    save_freq='epoch', 
    options=None)

tuner.search(X_train, y_train_oh, 
             epochs=100, 
             batch_size=800, 
             callbacks=[ClearTrainingOutput(), early_stop, save_best], 
             validation_split=0.15)

Trial 37 Complete [00h 00m 09s]
val_accuracy: 0.384010910987854

Best val_accuracy So Far: 0.49591365456581116
Total elapsed time: 00h 06m 12s

Search: Running Trial #38

Hyperparameter    |Value             |Best Value So Far 
units             |72                |48                
units1            |48                |72                
units2            |88                |40                
learning_rate     |0.01              |0.01              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 

Epoch 1/2
55/68 [=======================>......] - ETA: 0s - loss: 1.4383 - accuracy: 0.4342

KeyboardInterrupt: 

In [100]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_hps

In [101]:
model = tuner.hypermodel.build(best_hps)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 64)            4115456   
                                                                 
 lstm_1 (LSTM)               (None, 48)                21696     
                                                                 
 batch_normalization_1 (Batc  (None, 48)               192       
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 72)                3528      
                                                                 
 dense_4 (Dense)             (None, 40)                2920      
                                                                 
 dense_5 (Dense)             (None, 6)                 246       
                                                      

In [102]:
model.fit(X_train, y_train_oh, 
          epochs=100,
          batch_size=600,
          callbacks=[early_stop, save_best],
          validation_split=0.15)

Epoch 1/100
90/91 [============================>.] - ETA: 0s - loss: 1.3852 - accuracy: 0.4659
Epoch 00001: val_accuracy improved from -inf to 0.40654, saving model to best_a.hdf5
91/91 [==============================] - 5s 53ms/step - loss: 1.3854 - accuracy: 0.4659 - val_loss: 1.6945 - val_accuracy: 0.4065
Epoch 2/100
91/91 [==============================] - ETA: 0s - loss: 1.1645 - accuracy: 0.5666
Epoch 00002: val_accuracy improved from 0.40654 to 0.47590, saving model to best_a.hdf5
91/91 [==============================] - 5s 54ms/step - loss: 1.1645 - accuracy: 0.5666 - val_loss: 1.5543 - val_accuracy: 0.4759
Epoch 3/100
91/91 [==============================] - ETA: 0s - loss: 1.0900 - accuracy: 0.5959
Epoch 00003: val_accuracy improved from 0.47590 to 0.50220, saving model to best_a.hdf5
91/91 [==============================] - 5s 54ms/step - loss: 1.0900 - accuracy: 0.5959 - val_loss: 1.4693 - val_accuracy: 0.5022
Epoch 4/100
90/91 [============================>.] - ETA: 0s - l

In [103]:
# X_test_enc = token.texts_to_sequences(X_test)
# X_test = sequence.pad_sequences(X_test_enc, maxlen=15, padding='post')

In [104]:
model.load_weights(checkpoint_filepath)
test_loss, test_acc = model.evaluate(X_test,  y_test_oh, verbose=1)

351/351 [==============================] - 0s 926us/step - loss: 1.3190 - accuracy: 0.5071


In [105]:
from sklearn.metrics import classification_report

y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.82      0.63      1867
           1       0.64      0.42      0.51      1858
           2       0.48      0.50      0.49      1906
           3       0.55      0.34      0.42      1696
           4       0.47      0.44      0.46      2026
           5       0.46      0.51      0.48      1876

    accuracy                           0.51     11229
   macro avg       0.52      0.51      0.50     11229
weighted avg       0.52      0.51      0.50     11229

